In [36]:
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [37]:
import zarr
from candidate_graph import create_candidate_graph
import daisy
import linajea
from copy import deepcopy

In [38]:
candidate_graph = create_candidate_graph('anno_alice_T2030_tiny.zarr')

/
 ├── .zarr
 ├── Fragment_stats
 │   ├── Position
 │   ├── Volume
 │   └── id
 ├── Fragments (3, 15, 30, 30) uint64
 ├── Merge_tree
 │   ├── Merge
 │   └── Scoring
 ├── Raw (4, 3, 15, 30, 30) float64
 └── gt_trackimage (3, 15, 30, 30) uint64
 iterating one merge tree cost:  1434.5845341682434
 iterating one merge tree cost:  8465.805722236633
The candidate graph was saved and it cost: 8465.805779457092


In [ ]:
G = candidate_graph

In [ ]:
import networkx as nx

nx.write_graphml(candidate_graph, 'candidate.graphml')

In [50]:
from linajea_cost_test import get_merge_graph ,get_merge_graph_from_array
t = 0
z = zarr.open('anno_alice_T2030_tiny.zarr','r') 
merge_tree = z['Merge_tree/Merge/'+str(t)]
scores = z['Merge_tree/Scoring/'+str(t)]
merge_tree = get_merge_graph_from_array(merge_tree,scores)

g = get_merge_graph(G,0)

In [98]:
# check the root nodes 
import networkx as nx
for node in candidate_graph.nodes:
    if  candidate_graph.nodes[node]['parent'] == node:
        print(candidate_graph.nodes[node])


{'t': 0, 'z': 7, 'y': 14, 'x': 14, 'score': 0.9081573486328125, 'parent': 474989143487680000, 'id': 474989143487680000}
{'t': 1, 'z': 7, 'y': 14, 'x': 14, 'score': 0.8916463255882263, 'parent': 474989143487680001, 'id': 474989143487680001}
{'t': 2, 'z': 7, 'y': 14, 'x': 14, 'score': 0.9649198055267334, 'parent': 474989143487680002, 'id': 474989143487680002}


In [99]:
roi = daisy.Roi((0, 0, 0, 0), (3, 15, 25, 25))

    # input graph
graph = linajea.tracking.TrackGraph(G, frame_key='t', roi=roi)

In [100]:
from linajea_cost_test import get_edge_indicator_costs, get_node_indicator_costs, get_constraints


import time
import logging

logger = logging.getLogger(__name__)


class SolvingConfig():
    def __init__(self, solve_config):
        self.solve = solve_config

In [101]:
ps = {
            "track_cost": 4.0,
            "weight_edge_score": -0.1,
            "weight_node_score": -0.1,
            "selection_constant": -1.0,
            "max_cell_move": 0.0,
            "block_size": [5, 100, 100, 100],
            }
job = {"num_workers": 5, "queue": "normal"}

solve_config = linajea.config.SolveConfig(parameters=ps,
                                            job=job,
                                            context=[2, 100, 100, 100])

solve_config.solver_type = None
solve_config.timeout = 100000

In [102]:
config = SolvingConfig(solve_config)
constrs = get_constraints()

linajea.tracking.track(
                graph,
                config,
                frame_key='t',
                selected_key='selected',
                edge_indicator_costs=get_edge_indicator_costs,
                node_indicator_costs=get_node_indicator_costs,
                constraints_fns=constrs[0],
                pin_constraints_fns=constrs[1]
                )


Could not create Gurobi backend: Gurobi error in pylp/impl/solvers/GurobiBackend.cpp:22: No Gurobi license found (user silja, host Precision-5820-Tower, hostid e5051bc3, cores 12)
feasible solution found by trivial heuristic after 0.1 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       363 del vars, 737 del conss, 0 add conss, 187 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 589 clqs
(round 2, exhaustive) 363 del vars, 737 del conss, 0 add conss, 187 chg bounds, 0 chg sides, 0 chg coeffs, 17430 upgd conss, 0 impls, 616 clqs
(round 3, medium)     698 del vars, 16935 del conss, 32394 add conss, 187 chg bounds, 334 chg sides, 334 chg coeffs, 17430 upgd conss, 0 impls, 32982 clqs
(round 4, exhaustive) 698 del vars, 33132 del conss, 32394 add conss, 187 chg bounds, 334 chg sides, 655 chg coeffs, 17430 upgd conss, 0 impls, 32982 clqs
(round 5, medium)     791 del vars, 33225 del conss, 32394 add conss, 187 chg bounds, 334 chg sides, 655 chg coeffs, 17430 

In [103]:
nx.write_graphml(graph,'test_solver.graphml')

set the parameter from the tomal file. 
- TODO for grid search

In [86]:
from linajea.config import TrackingConfig
config_file = 'config.toml'
config = TrackingConfig.from_file(config_file)

config.solve.parameters

[SolveParametersConfig(track_cost=7, weight_node_score=-5, selection_constant=-1, weight_division=-11, division_constant=2.5, weight_child=2.0, weight_continuation=-1.0, weight_edge_score=10, cell_state_key=None, block_size=[15, 512, 512, 712], max_cell_move=45, roi=None, feature_func='noop', val=True, tag=None),
 SolveParametersConfig(track_cost=7, weight_node_score=-0.1, selection_constant=5, weight_division=-11, division_constant=2.5, weight_child=1.0, weight_continuation=-1.0, weight_edge_score=-1, cell_state_key=None, block_size=[15, 512, 512, 712], max_cell_move=45, roi=None, feature_func='noop', val=True, tag=None),
 SolveParametersConfig(track_cost=1, weight_node_score=-1, selection_constant=2, weight_division=-10, division_constant=2.5, weight_child=2.0, weight_continuation=-1.0, weight_edge_score=-1, cell_state_key=None, block_size=[15, 512, 512, 712], max_cell_move=45, roi=None, feature_func='noop', val=True, tag=None),
 SolveParametersConfig(track_cost=7, weight_node_score=

In [87]:


roi = daisy.Roi((0, 0, 0, 0), (10, 100, 100, 100))
graph = linajea.tracking.TrackGraph(G,frame_key='t',roi=roi)
constrs = get_constraints()

# get sovlver paremeters 
parameters = deepcopy(config.solve.parameters)


selected_keys = ['selected_' + str(i) for i in range(len(parameters))]


job = {'num_workers':24,'queue': 'normal'}

solve_config = linajea.config.SolveConfig(parameters=parameters,
                                            job=job,
                                            context = [10,100,100,100])
solve_config.timeout = 10000000
solve_config.solver_type = None
config = SolvingConfig(solve_config)
linajea.tracking.track(
            graph,
            config,
            frame_key='t',
            selected_key=selected_keys,
            edge_indicator_costs=get_edge_indicator_costs,
            node_indicator_costs=get_node_indicator_costs,
            constraints_fns=constrs[0],
            pin_constraints_fns=constrs[1]
            )

Could not create Gurobi backend: Gurobi error in pylp/impl/solvers/GurobiBackend.cpp:22: No Gurobi license found (user silja, host Precision-5820-Tower, hostid e5051bc3, cores 12)
feasible solution found by trivial heuristic after 0.1 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       363 del vars, 737 del conss, 0 add conss, 187 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 589 clqs
(round 2, exhaustive) 363 del vars, 737 del conss, 0 add conss, 187 chg bounds, 0 chg sides, 0 chg coeffs, 17430 upgd conss, 0 impls, 616 clqs
(round 3, medium)     699 del vars, 16936 del conss, 32394 add conss, 187 chg bounds, 334 chg sides, 334 chg coeffs, 17430 upgd conss, 0 impls, 32981 clqs
(round 4, exhaustive) 699 del vars, 33133 del conss, 32394 add conss, 187 chg bounds, 334 chg sides, 655 chg coeffs, 17430 upgd conss, 0 impls, 32981 clqs
(round 5, medium)     792 del vars, 33226 del conss, 32394 add conss, 187 chg bounds, 334 chg sides, 655 chg coeffs, 17430 

In [88]:
nx.write_graphml(graph,'solver.graphml')